In [2]:
import sys
sys.path.append('../../ABEL/')
from abel import *
import numpy as np
import time
import scipy.constants as SI 
import abel.utilities.plasma_physics as pp

%load_ext autoreload
%autoreload 2

In [8]:
# define driver
driver = SourceBasic()
driver.charge = -2.7e10 * SI.e # [C]
driver.energy = 31.25e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 50e-6 # [m]
driver.z_offset = 580e-6 # [m]
driver.emit_nx, driver.emit_ny = 80e-6, 80e-6 # [m rad]
driver.beta_x, driver.beta_y = 5, 5 #30e-3, 30e-3 # [m]
driver.num_particles = 100000
driver.jitter.x = 0 # [m] 2e-6
driver.jitter.y = 0 # [m] 100e-9
driver.jitter.t = 0 # 10e-15 # [m]
driver.symmetrize = True
driver.seed = 1
#drive_beam = driver.track()  # Convert a Source object to a Beam object.  ######


# define stage
stage = StageQuasistatic2d()
#stage = StagePrtclTransWakeInstability()                      #######
#stage.drive_beam = drive_beam                                 #######
stage.driver_source = driver
stage.length = 11 # [m]
stage.nom_energy_gain = 31.9e9/(11/stage.length) # [eV]
stage.plasma_density = 2e21 # [m^-3]
stage.ramp_beta_mag = 5
stage.radiation_reaction = False
stage.save_evolution = True
if stage.radiation_reaction == False:
    stage.save_evolution = False

# define first stage (half length)
first_stage = StageQuasistatic2d()
#first_stage = StagePrtclTransWakeInstability()                      #######
#first_stage.drive_beam = drive_beam                                 #######
first_stage.driver_source = driver
first_stage.nom_energy_gain = stage.nom_energy_gain/2
first_stage.length = stage.length/2
first_stage.plasma_density = stage.plasma_density
first_stage.ramp_beta_mag = stage.ramp_beta_mag
first_stage.radiation_reaction = stage.radiation_reaction
first_stage.save_evolution = stage.save_evolution

# define beam
source = SourceBasic()
source.charge = -1e10 * SI.e # [C]
source.energy = 5e9 # [eV]
#source.rel_energy_spread = 0.1
source.rel_energy_spread = 0.01
source.bunch_length = 10e-6 # [m]
source.z_offset = 0e-6 # [m]
source.emit_nx, source.emit_ny = 160e-6, 0.56e-6 # [m rad]
source.beta_x = stage.matched_beta_function(source.energy)
source.beta_y = source.beta_x
source.num_particles = 50000
source.jitter.t = driver.jitter.t
source.symmetrize = True
source.seed = driver.seed
#beam = source.track()


# define interstage
interstage = InterstageBasic()
#interstage = InterstageElegant()
interstage.beta0 = lambda E: stage.matched_beta_function(E)
interstage.dipole_length = lambda E: 1 * np.sqrt(E/10e9) # [m(eV)]
interstage.dipole_field = 1.  # [T]
#interstage.dipole_field = lambda E: np.min([1.0, 100e9/E]) # [T]

# beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_x, bds.beta_y = 8e-3, 0.4e-3 # [m]
bds.bunch_length = 0.75 * bds.beta_y

# define linac
linac = Linac()
linac.source = source
linac.stage = stage
linac.first_stage = first_stage
linac.interstage = interstage
linac.num_stages = 16
shots = 1

In [9]:
# scan the driver timing jitter
density_values = np.logspace(20, 23, 5)
linac.scan(f'scan_densities_RR_{stage.radiation_reaction}', 
           lambda obj, val: setattr(obj.stage, 'plasma_density', val) or setattr(obj.first_stage, 'plasma_density', val) or obj, 
           density_values,
           label='Relative energy spread [%]',
           scale=1e20,
           num_shots_per_step=5, 
           parallel=False,
           overwrite=True);

>> SHOT 1/25
Tracking element 1 (SourceBasic, stage 0, s = 0.0 m, 5.0 GeV, -1.60 nC, 1.0% rms, 157.6/0.6 µm-rad)
Tracking element 2 (StageQuasistatic2d, stage 1, s = 5.5 m, 3.8 GeV, -1.60 nC, 2.7% rms, 345.8/1.2 µm-rad)
Tracking element 3 (InterstageBasic, stage 1, s = 12.4 m, 3.8 GeV, -1.60 nC, 2.7% rms, 340.2/1.2 µm-rad)
Tracking element 4 (StageQuasistatic2d, stage 2, s = 23.4 m, 2.2 GeV, -1.60 nC, 12.0% rms, 368.7/1.3 µm-rad)
Tracking element 5 (InterstageBasic, stage 2, s = 34.4 m, 2.2 GeV, -1.60 nC, 12.0% rms, 369.2/1.3 µm-rad)
Tracking element 6 (StageQuasistatic2d, stage 3, s = 45.4 m, 1.0 GeV, -1.60 nC, 39.9% rms, 376.7/1.4 µm-rad)
Tracking element 7 (InterstageBasic, stage 3, s = 59.4 m, 1.0 GeV, -1.60 nC, 39.9% rms, 379.0/1.4 µm-rad)
Tracking element 8 (StageQuasistatic2d, stage 4, s = 70.4 m, 0.6 GeV, -1.02 nC, 55.7% rms, 399.2/1.5 µm-rad)
Tracking element 9 (InterstageBasic, stage 4, s = 86.7 m, 0.6 GeV, -1.02 nC, 55.7% rms, 384.2/1.5 µm-rad)


KeyboardInterrupt: 

In [ ]:
linac.plot_beam_function(Beam.rel_energy_spread, label='Energy spread (%)', scale=1e20, xscale='log')